In [1]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
import polars as pl
from concurrent.futures import ProcessPoolExecutor
import spacy
from functools import partial
import re
from PIL import Image

pl.Config.set_fmt_str_lengths(200)

/home/enric/projects/tiny_diffusion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


polars.config.Config

In [ ]:
PATH = '/mnt/sd1tb/tinydiffusion/dataset_v0/'

In [8]:
df = pl.read_parquet(PATH + 'dataset_raw.parquet')
len(df)

374

In [9]:
df = pl.read_parquet(PATH + 'imgs/00000.parquet')
len(df)

374

In [10]:
df

id,caption,url,key,status,error_message,width,height,original_width,original_height,exif,sha256
str,str,str,str,str,str,i32,i32,i32,i32,str,str
"""3846285008153""","""MP3 & FM Music Player with LCD Screen Clip-On Mini Support 32GB Micro SD TF-Assorted Colors - christmasgiftbuy""","""https://cdn.shopify.com/s/files/1/0048/2787/3383/products/LCD_screen_MP3_player_420x.jpg?v=1584136939""","""00000003""","""success""",null,256,256,420,420,"""{""Image Orientation"": ""Horizontal (normal)"", ""Image XResolution"": ""144"", ""Image YResolution"": ""144"", ""Image ResolutionUnit"": ""Pixels/Inch"", ""Image YCbCrPositioning"": ""Centered"", ""Image ExifOffset"": ""1…","""0e97f86680db0011b6dd05a420fd6c882ad5d8b275a223bd51d3f673f4ccf479"""
"""3836587000648""","""Victure MP3 Player Clip Bluetooth 8G with FM Radio Sport Pedometer Voice Recorder Earphones Expandable up to 128GB 3""","""https://m.media-amazon.com/images/I/41t12RZwfHL._SS600_.jpg""","""00000006""","""success""",null,256,256,600,600,"""{}""","""8cd1927f96ba9212283d4c3539b326d84a150452584e5415f980fa1085892ffb"""
"""4116514001738""","""23 Pcs Tool Set Kit Box Pink Women Ladies Girls Female Hand Tools Pliers • 16.11£""","""https://www.dealsanimg.com/d/l400/pict/383810886274_/23-pcs-tool-set-kit-box-pink-women-ladies-girls-female-hand.jpg""","""00000001""","""failed_to_resize""","""OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:813: error: (-215:Assertion failed) !buf.empty() in function 'imdecode_' """,null,null,null,null,null,null
"""2923832009258""","""Dialectical Behavior Therapy: The Ultimate Guide for Using DBT for Borderline Personality Disorder, Difficult Emotions and Mood Swings, Including Techniques such as Mindfulness and Emotion Regulation …","""https://m.media-amazon.com/images/I/511LtEZ2m5L._SL320_.jpg""","""00000008""","""success""",null,256,256,320,320,"""{}""","""8bb9efa5544352e2190728c851d3d61a96fea4bcd793134e34e30cdb76c8c366"""
"""3715889016426""","""Primary image for Let's Swing Disc Three [Audio CD] Stan Kenton; Ben Pollack; Cab Calloway; Ben..""","""https://images.bonanzastatic.com/afu/images/2375/2386/66/61ixzuadt1l._sl1500_.jpg""","""00000005""","""success""",null,256,256,500,480,"""{}""","""12fdf117cb0a551343ded61c6fde8308900affa584d186a85f1d4f11387e4f6e"""
…,…,…,…,…,…,…,…,…,…,…,…
"""2312043014249""","""Beautiful fall colored maple leaves, next to an aspen tree trunk, beneath a deep blue sky in Colorado""","""https://photos.smugmug.com/Stock-Photos-and-Illustrations/ScenicLandscape/i-W2WmNT9/2/4265efb1/M/trip%20176-M.jpg""","""00000372""","""success""",null,256,256,338,450,"""{""Image ExifOffset"": ""26""}""","""51308c0483ad2d901ac270a515ff7bab113a386839c1b6d8092b28c5e9887ed7"""
"""181432000232""","""magenta OASAP bag - navy Topshop jeans - black Lovelyshoes blazer""","""http://images3.chictopia.com/photos/RetroStreetStation/4845128544/navy-topshop-jeans-black-lovelyshoes-blazer-magenta-oasap-bag.jpg""","""00000328""","""success""",null,256,256,300,450,"""{""Image ImageWidth"": ""3456"", ""Image ImageLength"": ""5184"", ""Image BitsPerSample"": ""[8, 8, 8]"", ""Image PhotometricInterpretation"": ""2"", ""Image Make"": ""Canon"", ""Image Model"": ""Canon EOS 550D"", ""Image Ori…","""47de27ebe85ac16676b40e1ba2706643d340ab7d5ff5e69435965cbcb1bf99e9"""
"""1891417010526""","""display product reviews for roughneck storage shed common 5 ft x 6""","""https://mobileimages.lowes.com/product/converted/071691/071691481799.jpg""","""00000371""","""success""",null,256,256,900,900,"""{}""","""0b3a4d2e31fd77c56edfc49f9112fa69e0d5ebe99f700c4371998fff74d0ddef"""
